In [1]:
import torch
import pandas as pd


In [2]:
# Load the CSV file into a pandas DataFrame
df = pd.read_csv('/content/drive/MyDrive/Projects/zero/data/object_descriptions.csv')

In [3]:
from transformers import AutoTokenizer, CLIPTextModelWithProjection

model = CLIPTextModelWithProjection.from_pretrained("zer0int/LongCLIP-GmP-ViT-L-14")
#model = CLIPTextModelWithProjection.from_pretrained("openai/clip-vit-base-patch32")

tokenizer = AutoTokenizer.from_pretrained("zer0int/LongCLIP-GmP-ViT-L-14")

# Create a new DataFrame to store object names and embeddings
embeddings_df = pd.DataFrame(columns=['object_name', 'embedding'])

for index, row in df.iterrows():
    description = row['description']  #  'object_descriptions' is the column name
    object_name = row['object_name']  #  'object_descriptions' is the column name

    inputs = tokenizer([description], padding=True, return_tensors="pt")
    outputs = model(**inputs)
    embedding = outputs.text_embeds[0].detach().numpy() # Convert tensor to numpy array
    embeddings_df.loc[index] = [object_name, embedding]

# Save the new DataFrame to a CSV file
embeddings_df.to_csv('groundTruthEmbeddings.csv', index=False)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/746 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.71G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/907 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/961k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

Get embeddings for participant descriptions

In [5]:
# Load the CSV file into a pandas DataFrame
df = pd.read_csv('/content/drive/MyDrive/Projects/zero/data/zeroDescriptionsByBlock.csv')

In [12]:
# Add a new column 'embedding' to the DataFrame
df['embedding'] = df['embedding'].astype(object) # change the column type to 'object'
df

<generator object DataFrame.iterrows at 0x7e12e655e8a0>

In [16]:
for index, row in df.iterrows():
    description = row['Transcription']

    if pd.isna(description) or not description:  # Check if description is empty or NaN
        continue #skip row if empty

    # Truncate the description if it's longer than 248
    if len(description) > 248:
        description = description[:248]

    inputs = tokenizer([description], padding=True, return_tensors="pt")
    outputs = model(**inputs)
    embedding = outputs.text_embeds[0].detach().numpy()
    df.loc[index, 'embedding'] = embedding.tolist()

# Save the updated DataFrame (with embeddings)
df.to_csv('zeroDescriptionsByBlockEmbedded.csv', index=False)

ValueError: Must have equal len keys and value when setting with an iterable